In [ ]:
"""
config_cnn.py
---------------
Fichier de configuration pour notre CNN :
- chemins
- hyperparamètres d'entraînement
"""

# Hyperparamètres d'entraînement
BATCH_SIZE = 64
EPOCHS = 50
LEARNING_RATE = 1e-3
VALIDATION_SPLIT = 0.2

# Paramètres du modèle
INPUT_SHAPE = (32, 32, 3)   # CIFAR-10 : images 32x32 RGB
NUM_CLASSES = 10            # 10 classes

# Callbacks
EARLY_STOPPING_PATIENCE = 5   # nombre d'époques sans amélioration avant arrêt
LR_REDUCE_PATIENCE = 3        # nombre d'époques sans amélioration avant réduction du LR

# Chemin pour sauvegarder le meilleur modèle
BEST_MODEL_PATH = "data/best_cnn_cifar10.h5"


"""
model_cnn.py
-------------
Définit la fonction build_cnn_model() qui crée un modèle CNN Keras.
"""

import tensorflow as tf
from tensorflow.keras import layers, models
from config_cnn import INPUT_SHAPE, NUM_CLASSES, LEARNING_RATE

def build_cnn_model():
    """
    Crée et compile un modèle CNN pour la classification d'images.
    Retourne : un modèle Keras compilé.
    """

    model = models.Sequential()

    # Bloc 1 de convolution
    model.add(layers.Conv2D(32, (3, 3), activation="relu", padding="same",
                            input_shape=INPUT_SHAPE))
    model.add(layers.Conv2D(32, (3, 3), activation="relu", padding="same"))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.25))  # régularisation pour limiter l'overfitting

    # Bloc 2 de convolution
    model.add(layers.Conv2D(64, (3, 3), activation="relu", padding="same"))
    model.add(layers.Conv2D(64, (3, 3), activation="relu", padding="same"))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.25))

    # Bloc 3 de convolution
    model.add(layers.Conv2D(128, (3, 3), activation="relu", padding="same"))
    model.add(layers.Conv2D(128, (3, 3), activation="relu", padding="same"))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.25))

    # Passage en fully-connected
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation="relu"))
    model.add(layers.Dropout(0.5))   # Dropout plus fort sur la partie dense

    # Couche de sortie
    model.add(layers.Dense(NUM_CLASSES, activation="softmax"))

    # Compilation du modèle
    optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",  # labels comme entiers 0..9
        metrics=["accuracy"]
    )

    return model


"""
train_cnn.py
-------------
Script principal pour :
- Charger CIFAR-10
- Prétraiter les données
- Appliquer une augmentation de données (data augmentation)
- Entraîner le CNN avec callbacks (early stopping, reduction du LR, sauvegarde du meilleur modèle)
- Évaluer le modèle
"""

import tensorflow as tf
from tensorflow.keras import callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from config_cnn import (
    BATCH_SIZE,
    EPOCHS,
    VALIDATION_SPLIT,
    EARLY_STOPPING_PATIENCE,
    LR_REDUCE_PATIENCE,
    BEST_MODEL_PATH
)
from model_cnn import build_cnn_model

def load_data():
    """
    Charge le dataset CIFAR-10 depuis Keras.
    Renvoie : (x_train, y_train), (x_test, y_test)
    """
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

    # Normalisation des pixels dans [0, 1]
    x_train = x_train.astype("float32") / 255.0
    x_test = x_test.astype("float32") / 255.0

    # y_train et y_test sont des entiers (0..9), ce qui est compatible avec sparse_categorical_crossentropy
    return (x_train, y_train), (x_test, y_test)

def get_data_generator():
    """
    Crée un générateur d'images avec augmentation de données.
    Cette augmentation aide à limiter l'overfitting.
    """
    datagen = ImageDataGenerator(
        rotation_range=15,       # rotations aléatoires
        width_shift_range=0.1,   # décalage horizontal
        height_shift_range=0.1,  # décalage vertical
        horizontal_flip=True,    # flip horizontal
        zoom_range=0.1           # zoom léger
    )
    return datagen

def get_callbacks():
    """
    Crée les callbacks pour :
    - EarlyStopping : arrêter quand la val_loss ne s'améliore plus
    - ReduceLROnPlateau : diminuer le LR si la val_loss stagne
    - ModelCheckpoint : sauvegarder le meilleur modèle (val_loss minimale)
    """
    early_stopping = callbacks.EarlyStopping(
        monitor="val_loss",
        patience=EARLY_STOPPING_PATIENCE,
        restore_best_weights=True,
        verbose=1
    )

    reduce_lr = callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,       # diviser le LR par 2
        patience=LR_REDUCE_PATIENCE,
        min_lr=1e-6,
        verbose=1
    )

    checkpoint = callbacks.ModelCheckpoint(
        BEST_MODEL_PATH,
        monitor="val_loss",
        save_best_only=True,
        save_weights_only=False,
        verbose=1
    )

    return [early_stopping, reduce_lr, checkpoint]

def main():
    # 1. Charger les données
    (x_train, y_train), (x_test, y_test) = load_data()

    # 2. Créer le modèle CNN
    model = build_cnn_model()
    model.summary()  # affiche la structure du modèle

    # 3. Préparer le générateur de données avec augmentation
    datagen = get_data_generator()
    datagen.fit(x_train)

    # 4. Préparer les callbacks (early stopping, etc.)
    cb_list = get_callbacks()

    # 5. Entraîner le modèle
    #    On utilise flow(x, y) pour générer les batchs augmentés à la volée.
    history = model.fit(
        datagen.flow(x_train, y_train, batch_size=BATCH_SIZE),
        epochs=EPOCHS,
        validation_split=VALIDATION_SPLIT,  # ne fonctionne pas directement avec flow
        # Astuce : on peut split avant ou utiliser validation_data séparément.
        # Ici on fait un split manuel pour la validation.
        steps_per_epoch=int((1 - VALIDATION_SPLIT) * len(x_train) // BATCH_SIZE),
        callbacks=cb_list,
        verbose=1
    )

    # 6. Évaluer sur le jeu de test
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
    print(f"Performance sur le test : loss = {test_loss:.4f}, accuracy = {test_acc:.4f}")

    # 7. Sauvegarder le modèle final (optionnel, en plus du best_model)
    model.save("cnn_cifar10_final.h5")
    print("Modèle final sauvegardé sous 'cnn_cifar10_final.h5'.")

if __name__ == "__main__":
    main()



In [ ]:
"""
metrics_cnn.py
--------------
Calcule des métriques de classification pour un CNN Keras (CIFAR-10) :
Accuracy, Precision, Recall, F1-score, AUC-ROC (multi-classe) + matrice de confusion.
"""

import numpy as np
import tensorflow as tf
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    classification_report,
)
import matplotlib.pyplot as plt

# Noms CIFAR-10 (ordre standard)
CIFAR10_CLASS_NAMES = [
    "airplane", "automobile", "bird", "cat", "deer",
    "dog", "frog", "horse", "ship", "truck"
]

def load_cifar10():
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
    x_test = x_test.astype("float32") / 255.0
    y_test = y_test.reshape(-1)  # (N,1) -> (N,)
    return x_test, y_test

def plot_confusion_matrix(cm, class_names):
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation="nearest")
    plt.title("Matrice de confusion")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45, ha="right")
    plt.yticks(tick_marks, class_names)
    plt.xlabel("Prédictions")
    plt.ylabel("Vérité terrain")
    plt.tight_layout()
    plt.show()

def main():
    # 1) Charger le modèle entraîné (best ou final)
    # Ex : BEST_MODEL_PATH = "data/best_cnn_cifar10.h5"
    model_path = "data/best_cnn_cifar10.h5"
    model = tf.keras.models.load_model(model_path)

    # 2) Charger les données de test
    x_test, y_test = load_cifar10()

    # 3) Prédictions
    y_proba = model.predict(x_test, batch_size=128, verbose=0)  # shape (N,10)
    y_pred = np.argmax(y_proba, axis=1)

    # 4) Métriques
    acc = accuracy_score(y_test, y_pred)

    # Pour multi-classe : macro (toutes classes pareil) ou weighted (pondéré par support)
    prec_macro = precision_score(y_test, y_pred, average="macro", zero_division=0)
    rec_macro  = recall_score(y_test, y_pred, average="macro", zero_division=0)
    f1_macro   = f1_score(y_test, y_pred, average="macro", zero_division=0)

    prec_w = precision_score(y_test, y_pred, average="weighted", zero_division=0)
    rec_w  = recall_score(y_test, y_pred, average="weighted", zero_division=0)
    f1_w   = f1_score(y_test, y_pred, average="weighted", zero_division=0)

    # AUC-ROC multi-classe (One-vs-Rest)
    y_test_oh = tf.keras.utils.to_categorical(y_test, num_classes=y_proba.shape[1])
    auc_ovr_macro = roc_auc_score(y_test_oh, y_proba, multi_class="ovr", average="macro")

    cm = confusion_matrix(y_test, y_pred)

    print("=== Métriques CNN (CIFAR-10) ===")
    print(f"Accuracy : {acc:.4f}\n")

    print("---- Macro ----")
    print(f"Precision : {prec_macro:.4f}")
    print(f"Recall    : {rec_macro:.4f}")
    print(f"F1-score  : {f1_macro:.4f}\n")

    print("---- Weighted ----")
    print(f"Precision : {prec_w:.4f}")
    print(f"Recall    : {rec_w:.4f}")
    print(f"F1-score  : {f1_w:.4f}\n")

    print(f"AUC-ROC (OVR macro) : {auc_ovr_macro:.4f}\n")

    print("=== Rapport détaillé par classe ===")
    print(classification_report(y_test, y_pred, target_names=CIFAR10_CLASS_NAMES, zero_division=0))

    # 5) Matrice de confusion (affichage simple matplotlib)
    plot_confusion_matrix(cm, CIFAR10_CLASS_NAMES)

if __name__ == "__main__":
    main()
